In [1]:
from keras import backend as K
import time
import cv2
import os
import glob
import numpy as np
from numpy import genfromtxt
import tensorflow as tf
#import tqdm

/Users/s0c02nj/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
from multiprocessing.dummy import Pool
K.set_image_data_format('channels_first')

In [3]:
from fr_utils import *

In [4]:
from inception_blocks_v2 import *

In [5]:
PADDING = 50
#ready_to_detect_identity = True
FRmodel = faceRecoModel(input_shape=(3, 96, 96))

In [6]:
def img_to_array(image):
    img = cv2.resize(image, (96, 96)) 
    x_train = np.array([img])
    return x_train

In [7]:
def img_path_to_array(image_path):
    img1 = cv2.imread(image_path, 1)
    return img_to_array(img1)

In [8]:
def prepare_database_tr(location):
    database = {}

    # load all the images of individuals to recognize into the database
    for file in glob.glob(location):
        identity = os.path.splitext(os.path.basename(file))[0].strip('NISDCC-_6g')
        #print (identity)
        database[identity] = img_path_to_array(file)

    return database

In [9]:
train_image_arrays = prepare_database_tr('/Users/s0c02nj/Desktop/Dataset-AxisBankAI/final_database_sign/training/*.PNG')


In [10]:
triplet_str_tr=[]

t1= time.time()
for i in range(1,13):
    for ja in range(1,6):
        for k in range(23,52):
             for jn in range(1,6):
                    for jp in range(1,6):
                        #print (i,ja,k,jn,jp)
                        if (jp!=ja):
                            
                            if(i<10):
                                a='00'+ str(i)+'_00'+str(i)+'_00'+str(ja)
                                b='00'+ str(i) +'_00'+str(i)+'_00'+str(jp)
                                c='0'+str(k)+'_00'+str(i)+'_00'+str(jn)
                            else:
                                a='0'+str(i)+'_0'+str(i)+'_00'+str(ja)
                                b='0'+str(i)+'_0'+str(i)+'_00'+str(jp)
                                c='0'+str(k)+'_0'+str(i)+'_00'+str(jn)
                            
                            x= (a,b,c)
                            triplet_str_tr.append(x)
                        
                    
t2=time.time()

print (t2-t1)


0.123600006104


In [11]:
anchor_tr=[]
positive_tr=[]
negative_tr=[]

for img_name in triplet_str_tr:
    anchor_tr.append(train_image_arrays[img_name[0]])
    positive_tr.append(train_image_arrays[img_name[1]])
    negative_tr.append(train_image_arrays[img_name[2]])

In [12]:
x_a_tr=np.array(anchor_tr).reshape((len(anchor_tr), 3,96,96))
x_p_tr=np.array(positive_tr).reshape((len(positive_tr), 3,96,96))
x_n_tr=np.array(negative_tr).reshape((len(negative_tr), 3,96,96))


In [15]:
anc_emb_tr=FRmodel.predict(x_a_tr)
#pos_emb_tr=intermediate_layer_model_positive.predict([x_a_tr, x_p_tr, x_n_tr])
#neg_emb_tr=intermediate_layer_model_negative.predict([x_a_tr, x_p_tr, x_n_tr])

In [13]:
pos_emb_tr=FRmodel.predict(x_p_tr)

In [ ]:
neg_emb_tr=FRmodel.predict(x_n_tr)

In [ ]:
data_anc_tr = {}
data_pos_tr = {}
data_neg_tr = {}
for i in range(0,len(triplet_str_tr)):
    #Anchor dict
    a_id = triplet_str_tr[i][0]
    if a_id not in data_anc_tr.keys():
        data_anc_tr[a_id]= np.array(anc_emb_tr[i])
        
    #Pos dict
    p_id = triplet_str_tr[i][1]
    if p_id not in data_pos_tr.keys():
        data_pos_tr[p_id]= np.array(pos_emb_tr[i])
        
    #Neg dict
    n_id = triplet_str_tr[i][2]
    if n_id not in data_neg_tr.keys():
        data_neg_tr[n_id]= np.array(neg_emb_tr[i])

In [ ]:
#Pairwise pos str
pair_pos_str_tr=[]
for i in range(1,13):
    for ja in range(1,6):
        for jp in range(1,6):
            if (jp!=ja):
                if (i<10) :
                    a='00'+ str(i)+'_00'+str(i)+'_00'+str(ja)
                    b='00'+ str(i) +'_00'+str(i)+'_00'+str(jp)
                else :
                   a='0'+str(i)+'_0'+str(i)+'_00'+str(ja)
                   b='0'+str(i)+'_0'+str(i)+'_00'+str(jp)
                
                pos_pair=(a,b)
                pair_pos_str_tr.append(pos_pair)

In [ ]:
#Pairwise negative str
pair_neg_str_tr=[]
for i in range(1,13):
    for ja in range(1,6):
        for k in range(23,52):
            for jn in range(1,6):
                if (i<10):
                    a='00'+ str(i)+'_00'+str(i)+'_00'+str(ja)
                    b='0'+ str(k) +'_00'+str(i)+'_00'+str(jn)
                else :
                    a='0'+ str(i)+'_0'+str(i)+'_00'+str(ja)
                    b='0'+ str(k)+'_0'+str(i)+'_00'+str(jn)
                    
                pair_neg=(a,b)
                pair_neg_str_tr.append(pair_neg)
